In [1]:
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
from scipy.ndimage import zoom
import pandas as pd
from pipeline_setup import *
import time
from scipy.optimize import minimize

In [2]:
# Set file locations
advanced_settings_url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vTanjc08kc5vIN-icUMzMEGA9bJuDesLX8V_u2Ab6zSC4MOhLZ8Jrr18DL9o4ofKIrSq6FsJXhPWu3F/pub?gid=0&single=true&output=csv"

# Read in emissions from other sectors
sector_emissions_dict = set_sector_emissions_dict()
# Read in the advanced settings from the google sheet
params = read_advanced_settings(advanced_settings_url)
# Set the datablock
datablock_init = datablock_setup()

# Set the scenario parameters
params_baseline = set_baseline_scenario(params)

# Also add the baseline parameters to the datablock
datablock_init.update(params_baseline)

In [4]:
# Name the output parameters of the current calculator function 
# (ideally these would be set and returned by the calculator)
z1_name = "SSR weight"
z2_name = "SSR prot"
z3_name = "SSR fat"
z4_name = "SSR kcal"
z5_name = "emissions"
z6_name = "herd size"

z_names = [z1_name, z2_name, z3_name, z4_name, z5_name, z6_name]

# Name the params we are going to vary
names_x = ["ruminant",
           "foresting_pasture",
           "horticulture",
           "land_BECCS"]

# Set the bounds for the optimisation - to be the same as for the grid
# Make a 2d plot
x_mins = (-50, 5, 0, 0)
x_maxs = (50, 50, 200, 100)
x_bounds = bounds=[(x_mins[i], x_maxs[i]) for i in range(len(names_x)) ] # todo make this nd

In [5]:
# Create an class with anobjective function suitable for giving to the scipy minimizer
# Including a cache to avoid recomputing the same values
class FFCObjectiveWithCache:
    """A class to compute the objective function and constraints for FFC optimization with caching.
    Parameters:
        names_x (list): List of parameter names to be optimized.
        datablock_init (dict): Initial data block containing fixed parameters.
        params_default (dict): Default parameters for the optimization.
        sector_emissions_dict (dict): Dictionary containing emissions data for different sectors.
        verbosity (int): Level of verbosity for output messages.
    """
    def __init__(self, names_x, datablock_init, params_default, verbosity=0):
        self.names_x = names_x
        self.datablock_init = datablock_init
        self.params_default = params_default
        self._cache = {}
        self.verbosity = verbosity

        # Define the names of the z variables returned by the calculator
        self.z_names = ["SSR weight",
                   "SSR prot",
                   "SSR fat",
                   "SSR kcal",
                   "emissions",
                   "herd size"]

    def _calculate(self, x_tuple, verbosity):
        
        x = list(x_tuple)
        # Only recompute if not already cached
        
        if x_tuple not in self._cache:

            # Update the parameters with the current values
            params = self.params_default.copy()
            for i_name, name_string in enumerate(self.names_x):
                params[name_string] = x[i_name]
            
            # Perform the SSR and emissions calculation
            z_val = run_calculator(self.datablock_init, params)

            # cached dict
            zval_dict = {zn: zv for zn, zv in zip(self.z_names, z_val)}

            # Store the results in the cache
            self._cache[x_tuple] = zval_dict

            # Print out what's going on 
            if (verbosity > 1):
                for i_name, name_string in enumerate(self.names_x):
                    print(f"{name_string} = {x[i_name]:.10f}; ", end="")
                for i_name, name_string in enumerate(list(self._cache[x_tuple].keys())):
                    print(f"{name_string} = {self._cache[x_tuple][name_string]:.10f}; ", end="")
                print()

        return self._cache[x_tuple]

    # Define the objective function for minimization
    def objective(self, x, z_name_requested):
        x_tuple = tuple(x)
        return self._calculate(x_tuple, self.verbosity)[z_name_requested]

    # Define the objective function for SSR constraint
    def constraint_SSR(self, x, key, threshold):
        x_tuple = tuple(x)
        return self._calculate(x_tuple, self.verbosity)[key] - threshold
    

In [6]:
# Do a test calculation
z_val_baseline = run_calculator(datablock_init, params_baseline)
#print(f"SSR = {SSR_result:.8f}; GHG = {emissions_result:.8f}")

for zn, zval in zip(z_names, z_val_baseline):
    print(f"{zn} = {zval:.8f}; ", end="")
# This is my checksum for debugging: 
# SSR = 0.67366432; GHG = 94.22954995

SSR weight = 0.67366432; SSR prot = 0.73314955; SSR fat = 0.63337477; SSR kcal = 0.68543863; emissions = 94.77729174; herd size = 9151744.10648140; 

In [7]:
# Set tolerances, verbosity, and options for the minimizer
ffc_tol = 1e-6
options = {
    'disp': True,      # Show convergence messages
    'maxiter': 100,     # Max number of iterations
    'rhobeg' : 10 # Reasonable step size (mostly they are percentages, so change by 10%)
}


In [8]:
ffc_wrapper = FFCObjectiveWithCache(names_x, datablock_init, params_baseline, verbosity=2)

z_name_requested = "emissions"
x0 = [params_baseline[n] for n in names_x]
ffc_constraints = [{'type': 'ineq', 'fun': lambda x: ffc_wrapper.constraint_SSR(x, "SSR weight", threshold=0.7202)}]

result = minimize(
    lambda x: ffc_wrapper.objective(x, z_name_requested),
    x0,
    method='COBYLA',
    bounds=x_bounds,
    constraints=ffc_constraints,
    tol=ffc_tol,
    options=options
)

ruminant = 0.0000000000; foresting_pasture = 13.1700000000; horticulture = 0.0000000000; land_BECCS = 0.0000000000; SSR weight = 0.6736643225; SSR prot = 0.7331495528; SSR fat = 0.6333747730; SSR kcal = 0.6854386315; emissions = 94.7772917449; herd size = 9151744.1064814031; 
ruminant = 10.0000000000; foresting_pasture = 13.1700000000; horticulture = 0.0000000000; land_BECCS = 0.0000000000; SSR weight = 0.6728276606; SSR prot = 0.7317113340; SSR fat = 0.6317808633; SSR kcal = 0.6851689451; emissions = 96.0555846641; herd size = 9910109.0010977071; 
ruminant = 0.0000000000; foresting_pasture = 23.1700000000; horticulture = 0.0000000000; land_BECCS = 0.0000000000; SSR weight = 0.6160766353; SSR prot = 0.6633588104; SSR fat = 0.5484448061; SSR kcal = 0.6453296849; emissions = 77.0116113513; herd size = 6852993.6067655645; 
ruminant = 0.0000000000; foresting_pasture = 23.1700000000; horticulture = 10.0000000000; land_BECCS = 0.0000000000; SSR weight = 0.6225170535; SSR prot = 0.6623375483;

In [9]:
result

 message: Maximum number of function evaluations has been exceeded.
 success: False
  status: 2
     fun: 57.92405787851277
       x: [-5.000e+01  5.000e+00  2.000e+02  2.577e+01]
    nfev: 100
   maxcv: 2.211399119378399e-08

In [10]:
# The result is an OptimizeResult object
print("Optimization success:", result.success)
print("Message:", result.message)
print("Number of iterations:", result.nfev)
print("Optimal value of x:", result.x)
print("Minimum value of function:", result.fun)

Optimization success: False
Message: Maximum number of function evaluations has been exceeded.
Number of iterations: 100
Optimal value of x: [-49.999972     5.00000643 200.00000002  25.77151071]
Minimum value of function: 57.92405787851277


In [11]:
print(result['x'])      # Optimal parameters
print(result['fun'])    # Minimum value of the objective
print(result['success'])  # Boolean indicating if it was successful

[-49.999972     5.00000643 200.00000002  25.77151071]
57.92405787851277
False


In [12]:
# Display the results
#z1_val, z2_val = list(ffc_results.values())[:2]
z1_val = ffc_wrapper.objective(result.x, "SSR weight")
z2_val = ffc_wrapper.objective(result.x, "emissions")
print(f"SSR weight = {z1_val:.8f}; emissions = {z2_val:.8f}")


SSR weight = 0.72019998; emissions = 57.92405788
